In [1]:
import pandas as pd

In [5]:
#Import the csv file
csv = 'Resources/gun-violence-data_01-2013_03-2018.csv'
guns_db = pd.read_csv(csv)


In [6]:
guns_db.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,...,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,...,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


In [7]:
# Take out only the four columns that we would like to use
guns_db = guns_db[['date', 'state', 'n_killed', 'n_injured']]
guns_db.head()

,date,state,n_killed,n_injured
0,2013-01-01,Pennsylvania,0,4
1,2013-01-01,California,1,3
2,2013-01-01,Ohio,1,3
3,2013-01-05,Colorado,4,0
4,2013-01-07,North Carolina,2,2


In [8]:
#Filter out only years 2016 and 2017
guns_db = guns_db[(guns_db['date']>'2015-12-31') & (guns_db['date']<'2018-01-01')]

In [9]:
#Expand the date column to just keep the year
guns_db[['year','month', 'day']] = guns_db.date.str.split("-",expand=True)
guns_db.head()

,date,state,n_killed,n_injured,year,month,day
105711,2016-01-01,Massachusetts,0,1,2016,01,01
105712,2016-01-01,Connecticut,0,0,2016,01,01
105713,2016-01-01,Missouri,0,1,2016,01,01
105714,2016-01-01,Missouri,0,1,2016,01,01
105715,2016-01-01,Wisconsin,2,0,2016,01,01


In [10]:
# Group by year and state and the aggregate the n_killed and n_injuted columns
new_db = guns_db.groupby(['state', 'year']).agg('sum').sort_values(["year"], ascending=True)
new_db.head()

,,n_killed,n_injured
state,year,,
Alabama,2016,488,761
Maine,2016,18,37
Maryland,2016,429,748
Massachusetts,2016,112,468
Michigan,2016,385,700


In [11]:
#Reset index to state
new_db.reset_index('year', inplace=True)
new_db.head()

,year,n_killed,n_injured
state,,,
Alabama,2016,488,761
Maine,2016,18,37
Maryland,2016,429,748
Massachusetts,2016,112,468
Michigan,2016,385,700


In [12]:
#Split the 2016 into a new df
guns_2016 = new_db.loc[new_db['year']=='2016']

In [13]:
#Split the 2017 into a new df
guns_2017 = new_db.loc[new_db['year']=='2017']

In [14]:
#Drop the year column since the dfs are divided into two by the year
guns_2016 = guns_2016.drop(columns=['year'])
guns_2016.head()

,n_killed,n_injured
state,,
Alabama,488,761
Maine,18,37
Maryland,429,748
Massachusetts,112,468
Michigan,385,700


In [15]:
#Drop the year column since the dfs are divided into two by the year
guns_2017 = guns_2017.drop(columns=['year'])
guns_2017.head()

,n_killed,n_injured
state,,
Vermont,9,11
Connecticut,89,293
Oregon,99,170
Pennsylvania,635,1188
Colorado,199,300


In [ ]:
#Save the dataframes into new clean csv files
guns_2016.to_csv('2016_gun_violence.csv')
guns_2017.to_csv('2017_gun_violence.csv')